# IG to Test Kit FULL pipeline

## Setup

### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [5]:
import inspect
import json
import llm_utils
import importlib

## Initializing LLM Clients

In [6]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [3]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x1164010c0>}

## Text Extraction

### HTML to Markdown Conversion Using Markdownify (Langchain Tool)

In [4]:
import HTML_extractor

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
urls = [
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/index.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/ChangeHistory.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/examples.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/implementation.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/profiles.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/artifacts.html",
    "https://hl7.org/fhir/us/davinci-pdex-plan-net/CapabilityStatement-plan-net.html"
]

HTML_extractor.convert_urls_to_markdown(urls, output_dir="text_extraction/PlanNet/site/markdown")

Fetching pages: 100%|##########| 7/7 [00:00<00:00,  7.93it/s]


Created: index.md
Created: ChangeHistory.md
Created: examples.md
Created: implementation.md
Created: profiles.md
Created: artifacts.md
Created: CapabilityStatement_plan_net.md


### Markdown Post-processing

In [7]:
import markdown_cleaner
markdown_cleaner.process_directory("checkpoints/text_extraction/PlanNet/site/markdown", "checkpoints/post_processing/")

Found 7 markdown files in checkpoints/text_extraction/PlanNet/site/markdown
Cleaned and saved: checkpoints/post_processing/implementation.md
Cleaned and saved: checkpoints/post_processing/examples.md
Cleaned and saved: checkpoints/post_processing/profiles.md
Cleaned and saved: checkpoints/post_processing/ChangeHistory.md
Cleaned and saved: checkpoints/post_processing/artifacts.md
Cleaned and saved: checkpoints/post_processing/index.md
Cleaned and saved: checkpoints/post_processing/CapabilityStatement_plan_net.md

Processing complete: 7 files successfully cleaned, 0 failed


## Requirements Extraction

### Prompt-based Requirement Extraction

In [14]:
importlib.reload(reqs_extraction)

INFO:root:Current working directory: /Users/jrockhill/ONCLAIVE/pipeline
INFO:root:Project root: /Users/jrockhill/ONCLAIVE
INFO:root:Default markdown directory: /Users/jrockhill/ONCLAIVE/full-ig/markdown_cleaned
INFO:root:Default output directory: /Users/jrockhill/ONCLAIVE/reqs_extraction/initial_reqs_output
INFO:root:Prompt environment set up at: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Using prompts directory: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Requirements extraction prompt: /Users/jrockhill/ONCLAIVE/prompts/requirements_extraction.md


<module 'reqs_extraction' from '/Users/jrockhill/ONCLAIVE/pipeline/reqs_extraction.py'>

In [7]:
import reqs_extraction


In [ ]:
reqs_extraction.run_requirements_extractor('checkpoints/post_processing', 'checkpoints/requirements_extraction/markdown', 'claude', llm_clients)

### RAG-based Requirement Extraction

This extraction requirement extraction method differs from the first in that, as a part of the creation of its prompt, it performs a semantic search on example sections of FHIR IG text and the human-generated requirements that were produced in reference to those sections of text to find the most similar section(s) of FHIR IG text in the database and their associated requirement(s). Those sets of IG text and requirement(s) are then supplied to the LLM as few-shot examples

In [3]:
import rag_reqs_extraction

In [ ]:
importlib.reload(rag_reqs_extraction)
rag_reqs_extraction.full_pass(llm_clients, 'claude', "checkpoints/post_processing/", "checkpoints/requirements_extraction/RAG")

## Requirement Downselection

In [24]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["checkpoints/requirements_extraction/claude_reqs_list_v1_20250429_081756.md"],
    rag_files=["checkpoints/requirements_extraction/RAG/plan_net_reqs.json"]
    )

## Test Plan Generation

In [31]:
import logging
llm_clients.logger.setLevel(logging.INFO)

In [46]:
import req_to_testplan
importlib.reload(req_to_testplan)

req_to_testplan.generate_consolidated_test_plan(llm_clients, 'claude', llm_clients.logger, "checkpoints/requirements_downselect/filtered_requirements.json", "../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md", output_dir='checkpoints/testplan_generation')

INFO:llm_utils:Starting test plan generation with claude for FHIR Implementation Guide
INFO:llm_utils:Parsed 6 requirements from checkpoints/requirements_downselect/filtered_requirements.json
INFO:llm_utils:Parsed capability statement from ../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement 91782b26-ac7b-4ea8-ae82-629630ee21c2 using claude...
INFO:llm_utils:Identifying group for requirement ddcdfc94-021b-4e90-8ef9-556ff9ee933b using claude...
INFO:llm_utils:Identifying group for requirement 8b8400c1-a7dd-4404-af52-95ec11f02223 using claude...
INFO:llm_utils:Identifying group for requirement 4c406817-5996-496e-bcdd-5e8a6c803900 using claude...
INFO:llm_utils:Identifying group for requirement 7c480643-a168-4b03-ab5f-fba03a6eb989 using claude...
INFO:llm_utils:Identifying group for requirement c4dbd4a7-d5c8-47b7-8eb4-528f35d56309 using claude...
INFO:llm_utils:Requirements grouped into 6 categories
INFO:llm_utils:Group 'Security': 

{'requirements_count': 6,
 'group_count': 6,
 'test_plan_path': 'checkpoints/testplan_generation/claude_test_plan_20250609_153303.md'}

## Test Kit Generation

In [43]:
import plan_to_tests
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'claude',
    'checkpoints/testplan_generation/claude_test_plan_20250605_014638.md',
    '../test_kit_dev/inferno-guidance.md',
    output_dir='checkpoints/testkit_generation',
    expected_actors=["Health Plan API Actor", "Application Actor"]
)

Found 0 potential requirements


{'total_sections': 0,
 'total_requirements': 0,
 'generated_tests': 0,
 'module_dir': 'checkpoints/testkit_generation/plannet',
 'module_file': 'checkpoints/testkit_generation/plannet.rb'}